In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
#import xlrd
import openpyxl
from scipy.stats import multivariate_normal # used in gaussian pdf computation on mnist data.


#pip install openpyxl==3.0.5


In [2]:
data =pd.read_excel(r'C:/.../orgzn_data/orgzn_ratings.xlsx', engine='openpyxl') # import digits data

display(data)

,exec_key_matrices_overall,Innovation_leader,Regulatory_leader,Interoperability_leader,Network_and_Data_Security_leader,Population_Health_leader,Rev_Cycle_Management_leader,PAS_leader,Business_intelligence_leader
0,8.0,5.0,5.0,5.0,5.0,5.0,NaN,7.0,NaN
1,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7027,7.0,7.0,8.0,7.0,9.0,NaN,NaN,NaN,NaN
7028,7.0,7.0,7.0,7.0,7.0,3.0,NaN,NaN,NaN
7029,7.0,7.0,8.0,6.0,7.0,7.0,NaN,NaN,NaN
7030,9.0,8.0,9.0,6.0,8.0,8.0,NaN,NaN,NaN


In [3]:
data.shape  # (7032, 9)

(7032, 9)

In [4]:
'Missing value treatment'

print("\nMissing Values % in Data:\n{}".format(data.isnull().sum(axis = 0)/data.shape[0]))

'remove NAs and drop columns with almost every observation as NA'
data_1 = data.dropna(axis=0, how='any', thresh=None, subset= data.columns[:-2], inplace=False).iloc[:, 0:-2]
print("\nShape of data:-\n",data_1.shape)
#PAS_leader and Business_intelligence_leader have lots of null values. Drop these two columns. 
#data_1 = data_1. # remove last two columns as they are completely NAs.


'percentage of null values after their treatment'

print("\nMissing Values % in Data_1:\n{}".format(data_1.isnull().sum(axis = 0)/data_1.shape[0]))


Missing Values % in Data:
exec_key_matrices_overall           0.086320
Innovation_leader                   0.352531
Regulatory_leader                   0.366752
Interoperability_leader             0.356940
Network_and_Data_Security_leader    0.380262
Population_Health_leader            0.468999
Rev_Cycle_Management_leader         0.564420
PAS_leader                          0.926479
Business_intelligence_leader        0.984642
dtype: float64

Shape of data:-
 (2656, 7)

Missing Values % in Data_1:
exec_key_matrices_overall           0.0
Innovation_leader                   0.0
Regulatory_leader                   0.0
Interoperability_leader             0.0
Network_and_Data_Security_leader    0.0
Population_Health_leader            0.0
Rev_Cycle_Management_leader         0.0
dtype: float64


In [5]:
'Transform features and response to array shape'

'''
There are ten rating categories on a scale of 1-10 in the response variable.Further to it, there are other six variables, 
namely, Innovation_leader, Regulatory_leader, Interoperability_leader, Network_and_Data_Security_leader,
Population_Health_leader,Rev_Cycle_Management_leader, PAS_leader, Business_intelligence_leader which contributes to overall
organization rating(exec_key_matrices_overall).

'''
#response var/dependent variable - this is overall rating 
dv= data_1['exec_key_matrices_overall'] 
dv_arr = np.array(dv)
print("\noverall ratings array (response variable):")
display(dv_arr)

#independent var
ivs = data_1.iloc[: , 1:]  # consider all columns except the dependent variabe(dv) as independent variable
features = ivs.iloc[:, :].values 
#features_arr = np.array(features) #Features Array
print("\nFeatures array:")
display(features)


overall ratings array (response variable):


array([7., 2., 7., ..., 4., 8., 8.])


Features array:


array([[7., 7., 7., 8., 7., 7.],
       [3., 6., 6., 7., 3., 5.],
       [7., 7., 7., 7., 7., 7.],
       ...,
       [5., 5., 5., 5., 5., 5.],
       [7., 9., 8., 8., 8., 7.],
       [9., 8., 8., 8., 8., 7.]])

In [6]:
'Distribution of rating labels[1-10] in dependent variable i.e. overall rating'

c = dict(sorted(dv.value_counts().to_dict().items())) 
#c= dict(sorted(c.items()))
print("\nlabel distribution in dependent variable:\n {}".format(c))


label distribution in dependent variable:
 {1.0: 31, 2.0: 27, 3.0: 133, 4.0: 82, 5.0: 202, 6.0: 261, 7.0: 1051, 8.0: 604, 9.0: 265}


In [7]:

"get the prior probability distribution from the data"
prior_prob = np.ones(len(c)) # 10 since we have 10 class labels in our data.
for i in range(len(c)):
    prior_prob[i] = c[i+1]/dv.shape[0] #dictionary start with 1 not with 0.
prior_prob
prior_prob_list = list(prior_prob)
print("\nPrior Probability Distribution of classes:\n{}".format(list(zip(c.keys(),prior_prob_list))))

"2- create function to form the gaussian distribution per class label"

'''
To define a gaussian distribution, we would need mean and variance parameters.Calculate mean and variance for
each class subset of data.
'''
def normdist(data):  
    var = np.cov(data.T)  # to compute covariance, we need to horizontally stack variables.
    mean_array = []
    for i in range(data.shape[1]):
        mu = np.mean(data[:,i])
        mean_array.append(mu)
    
    ma = np.array(mean_array) #mean array from subset of data having a particular class label.
    print("\nmean array-")
    display(ma)
    print("\nvariance matrix-")
    display(var)
    distribution = multivariate_normal(ma, var)
    return distribution


Prior Probability Distribution of classes:
[(1.0, 0.011671686746987951), (2.0, 0.01016566265060241), (3.0, 0.050075301204819275), (4.0, 0.030873493975903613), (5.0, 0.07605421686746988), (6.0, 0.09826807228915663), (7.0, 0.3957078313253012), (8.0, 0.22740963855421686), (9.0, 0.09977409638554217)]


In [8]:
'Obtain the probability distribution of observations across all class labels in the features data'

labels = list(c.keys())
predictedvalues_traindata = pd.DataFrame()
for i in range(9):
    X_class = np.array(features[dv == labels[i]]) 
    print("\nlabel flag for which to computer multivariate distribution: ",i+1)
    norm_dist=normdist(X_class)
    predictions_class = prior_prob_list[i]*norm_dist.pdf(features) 
    
    '''
    The probability of all of the observations in features array (which is having 2656 obs) belonging to the 
    class label=1 or any other class label is computed by prior probability of a particular class multiplied by
    norm_dist.pdf(features_arr) which renders a probability value for each observation in features data associated 
    with a particular class. 
    
    '''
    predictedvalues_traindata = pd.concat([predictedvalues_traindata, pd.DataFrame(predictions_class)], axis = 1)

print('Probability distribution of observations across all class labels in the data-')
predictedvalues_traindata.columns = ["1", "2", "3", "4", "5", "6", "7", "8", "9"] 
display(predictedvalues_traindata)


label flag for which to computer multivariate distribution:  1

mean array-


array([2.12903226, 2.4516129 , 2.38709677, 3.48387097, 3.03225806,
       1.83870968])


variance matrix-


array([[3.3827957 , 2.50645161, 1.91505376, 2.00215054, 2.12903226,
        0.72150538],
       [2.50645161, 3.45591398, 1.81935484, 2.67419355, 2.6516129 ,
        1.60860215],
       [1.91505376, 1.81935484, 3.11182796, 1.97311828, 2.02043011,
        1.16451613],
       [2.00215054, 2.67419355, 1.97311828, 4.72473118, 3.4172043 ,
        1.94731183],
       [2.12903226, 2.6516129 , 2.02043011, 3.4172043 , 3.5655914 ,
        1.50537634],
       [0.72150538, 1.60860215, 1.16451613, 1.94731183, 1.50537634,
        2.67311828]])


label flag for which to computer multivariate distribution:  2

mean array-


...

array([8.55471698, 8.49811321, 8.35849057, 8.49056604, 8.18113208,
       7.59245283])


variance matrix-


array([[0.67218411, 0.3855203 , 0.36856775, 0.25335906, 0.39535449,
        0.51859634],
       [0.3855203 , 0.94033734, 0.5025729 , 0.45168668, 0.5268582 ,
        0.65453116],
       [0.36856775, 0.5025729 , 0.88236135, 0.39922813, 0.43860778,
        0.43453402],
       [0.25335906, 0.45168668, 0.39922813, 0.6675243 , 0.31610921,
        0.47720126],
       [0.39535449, 0.5268582 , 0.43860778, 0.31610921, 1.3231275 ,
        0.91879646],
       [0.51859634, 0.65453116, 0.43453402, 0.47720126, 0.91879646,
        2.60600343]])

Probability distribution of observations across all class labels in the data-


,1,2,3,4,5,6,7,8,9
0,5.108540e-09,1.116333e-07,1.849559e-07,7.128670e-07,6.659887e-06,2.128361e-05,3.079790e-04,3.871708e-04,7.697831e-05
1,1.090230e-10,4.953294e-09,3.148156e-07,2.187107e-07,1.902905e-07,2.450514e-07,3.309587e-07,1.210784e-10,7.528153e-16
2,3.168571e-09,1.213308e-07,2.449083e-07,1.506575e-06,9.507850e-06,3.232948e-05,4.006970e-04,4.293128e-04,5.375391e-05
3,6.974245e-11,1.208478e-08,4.035059e-09,9.583467e-08,1.646815e-07,9.558212e-07,4.680479e-05,1.612992e-04,2.063840e-04
4,1.331059e-10,1.909961e-08,2.928436e-07,1.776657e-07,2.582055e-07,1.055208e-07,1.668037e-07,1.195353e-10,3.602941e-18
...,...,...,...,...,...,...,...,...,...
2651,6.338815e-08,1.323396e-07,1.289208e-06,9.606569e-08,8.164453e-07,4.455722e-07,5.906738e-07,4.481004e-08,1.057606e-11
2652,3.026559e-12,2.233470e-09,9.353191e-10,1.718575e-08,2.057358e-08,3.268852e-07,1.558158e-06,6.205806e-07,6.543586e-08
2653,3.490336e-07,5.535480e-07,7.266873e-06,8.509856e-06,4.383230e-05,4.174862e-05,8.487271e-05,5.319942e-06,5.905765e-10
2654,1.295224e-09,3.122057e-08,3.296048e-08,2.284518e-07,1.268605e-06,4.238462e-06,1.094390e-04,1.241001e-04,2.952967e-05


In [9]:
#use features_arr of test data to predict probs on test features

In [10]:
'get the maximum probability for each observation to belong to a particular class'

'''
retrieve the maximum value out from each row/observation and check its respective class label(column) to get the overall
rating 
'''
# assign column names = class labels in our data
predictedvalues_traindata = predictedvalues_traindata.idxmax(axis=1, skipna=True) # get the max value in row and column name corresponding to that maximum rowvalue.


predictedvalues_traindata1 = list(predictedvalues_traindata)
predictedvalues_traindata1 = [int(i) for i in predictedvalues_traindata1]
temp= pd.DataFrame(list(zip(list(predictedvalues_traindata1), list(dv))), columns =['predicted_rating', 'actual_overallrating'])
print("Actual overall ratings and Predicted Ratings-")
display(temp)


#temp['indexval'] = ind

corr_data= round(float(temp.corr(method ='pearson').iloc[1][[0]]),2)
print("Correlation between actual rating and predicted ratings: ",corr_data)


print("\ncorrelation between actual and predictions-")
pd.crosstab(temp['actual_overallrating'], temp['predicted_rating'], rownames=['actual_overallrating'], colnames=['predicted_rating'])


Actual overall ratings and Predicted Ratings-


,predicted_rating,actual_overallrating
0,8,7.0
1,7,2.0
2,8,7.0
3,9,9.0
4,3,3.0
...,...,...
2651,3,3.0
2652,7,7.0
2653,7,4.0
2654,8,8.0


Correlation between actual rating and predicted ratings:  0.68

correlation between actual and predictions-


predicted_rating,1,2,3,4,5,6,7,8,9
actual_overallrating,,,,,,,,,
1.0,17,1,9,0,3,0,0,1,0
2.0,0,10,10,1,1,0,3,2,0
3.0,3,5,59,3,29,4,27,3,0
4.0,1,0,16,10,16,5,32,2,0
5.0,0,3,26,1,33,9,118,12,0
6.0,0,3,12,5,22,18,181,18,2
7.0,0,8,23,10,26,18,623,286,57
8.0,0,2,4,1,6,10,192,250,139
9.0,0,0,2,0,0,0,44,51,168


In [ ]:
'''
We observe that the matrix is dense diagonally which implies that most of our actual and predicted rating re similar.
'''

In [11]:
#https://peterroelants.github.io/posts/multivariate-normal-primer/